In [1]:
#Clean The Data
import csv
import os
import pandas as pd
import numpy as np
import functools
#Join Conditions
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)
#Kelvin to Fahrenheit conversion
def kelvin_to_fahrenheit(tempereture_kelvin):
   tempereture_fahrenheit = (((tempereture_kelvin - 273) * 9/5) + 32)
   return tempereture_fahrenheit

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/Raw_Data/')

Trip_Data = pd.read_csv('FHV_2018_Raw_Jan.csv', parse_dates=[0], low_memory=False)
Weather_Data = pd.read_csv('NYC_Weather.csv', parse_dates=[0], low_memory=False)
Zip_Data = pd.read_csv("fhv_zone_lookup_zipcode.csv", low_memory=False)

print('Import the csvs')


Import the csvs


In [2]:
Trip_Data['Pickup_DateTime'] = pd.to_datetime(Trip_Data['Pickup_DateTime'])
Trip_Data['DropOff_datetime'] = pd.to_datetime(Trip_Data['DropOff_datetime'])
Trip_Data['PUlocationID'] = pd.to_numeric(Trip_Data['PUlocationID'])
Trip_Data['DOlocationID'] = pd.to_numeric(Trip_Data['DOlocationID'])
Trip_Data['Pickup_DateTimeD'] = Trip_Data.Pickup_DateTime.dt.round('D')
Trip_Data['Hour'] = Trip_Data['Pickup_DateTime'].apply(lambda x: x.hour)

print('Clean the Trip Data')
Trip_Data.head()


Clean the Trip Data


,#,index,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Trip_Duration,Month,Pickup_DateTimeD,Hour
0,947310,2335653,2018-12-01 00:00:00,2018-12-01 00:41:27,4.0,80.0,1.0,B02800,0.0,41.0,12,2018-12-01,0
1,947311,5032538,2018-12-01 00:00:24,2018-12-01 00:48:55,142.0,17.0,0.0,B02764,0.0,49.0,12,2018-12-01,0
2,947312,4448659,2018-12-01 00:00:28,2018-12-01 00:09:10,232.0,79.0,0.0,B02395,0.0,9.0,12,2018-12-01,0
3,947313,2014476,2018-12-01 00:00:53,2018-12-01 00:16:45,162.0,107.0,0.0,B02869,0.0,16.0,12,2018-12-01,0
4,947314,2916146,2018-12-01 00:00:56,2018-12-01 00:29:09,213.0,69.0,1.0,B02871,0.0,28.0,12,2018-12-01,0


In [3]:
Weather_Data['dt_iso'] = pd.to_datetime(Weather_Data['dt_iso'])
Weather_Data['dt_isoD'] = Weather_Data.dt_iso.dt.round('D')
Weather_Data['Hour'] = Weather_Data['dt_iso'].apply(lambda x: x.hour)
Weather_Data = Weather_Data.fillna(0)

print('Clean the Weather Data')
Weather_Data.head()


Clean the Weather Data


,dt,cc,dt_iso,city_id,city_name,lat,lon,temp,temp_min,temp_max,...,snow_3h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon,dt_isoD,Hour
0,1356998400,1,2013-01-01 00:00:00,5128581,"New York City, New York",40.7306,-73.9867,272.03,270.37,273.15,...,0.0,0.0,0.0,20,801,Clouds,few clouds,02n,2013-01-01,0
1,1357002000,1,2013-01-01 01:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.46,270.37,272.15,...,0.0,0.0,0.0,20,801,Clouds,few clouds,02n,2013-01-01,1
2,1357005600,1,2013-01-01 02:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.19,269.82,272.15,...,0.0,0.0,0.0,20,801,Clouds,few clouds,02n,2013-01-01,2
3,1357009200,1,2013-01-01 03:00:00,5128581,"New York City, New York",40.7306,-73.9867,271.07,269.82,272.15,...,0.0,0.0,0.0,20,801,Clouds,few clouds,02n,2013-01-01,3
4,1357012800,1,2013-01-01 04:00:00,5128581,"New York City, New York",40.7306,-73.9867,270.83,269.82,272.15,...,0.0,0.0,0.0,1,800,Clear,sky is clear,01n,2013-01-01,4


In [4]:
Zip_Data['LocationID'] = pd.to_numeric(Zip_Data['LocationID'])
Zip_Data = Zip_Data.fillna(0)

print('Clean the Zip Data')
Zip_Data.head()


Clean the Zip Data


,LocationID,Borough,Zone,service_zone,zipcode
0,1,EWR,Newark Airport,EWR,7114
1,2,Queens,Jamaica Bay,Boro Zone,11366
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,10462
3,4,Manhattan,Alphabet City,Yellow Zone,10009
4,5,Staten Island,Arden Heights,Boro Zone,10306


In [5]:
Combined_Data = pd.merge(Trip_Data, Weather_Data, how='left', left_on=['Pickup_DateTimeD', 'Hour'], right_on=['dt_isoD', 'Hour'])

Combined_Data = pd.merge(Combined_Data, Zip_Data, how='left', left_on=['PUlocationID'], right_on=['LocationID'])

Combined_Data = pd.merge(Combined_Data, Zip_Data, how='left', left_on=['DOlocationID'], right_on=['LocationID'])

Combined_Data = Combined_Data[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'SR_Flag', 'Trip_Duration', 'city_name', 'temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description', 'Borough_x', 'Zone_x', 'zipcode_x', 'Borough_y', 'Zone_y', 'zipcode_y']]

print('Combine the data sets')
Combined_Data.head()


Combine the data sets


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Trip_Duration,city_name,temp,humidity,wind_speed,weather_id,weather_main,weather_description,Borough_x,Zone_x,zipcode_x,Borough_y,Zone_y,zipcode_y
0,2018-12-01 00:00:00,2018-12-01 00:41:27,4.0,80.0,1.0,41.0,"New York City, New York",277.31,79.0,3.0,500.0,Rain,light rain,Manhattan,Alphabet City,10009.0,Brooklyn,East Williamsburg,11216.0
1,2018-12-01 00:00:24,2018-12-01 00:48:55,142.0,17.0,0.0,49.0,"New York City, New York",277.31,79.0,3.0,500.0,Rain,light rain,Manhattan,Lincoln Square East,10024.0,Brooklyn,Bedford,11216.0
2,2018-12-01 00:00:28,2018-12-01 00:09:10,232.0,79.0,0.0,9.0,"New York City, New York",277.31,79.0,3.0,500.0,Rain,light rain,Manhattan,Two Bridges/Seward Park,10024.0,Manhattan,East Village,10024.0
3,2018-12-01 00:00:53,2018-12-01 00:16:45,162.0,107.0,0.0,16.0,"New York City, New York",277.31,79.0,3.0,500.0,Rain,light rain,Manhattan,Midtown East,10024.0,Manhattan,Gramercy,10024.0
4,2018-12-01 00:00:56,2018-12-01 00:29:09,213.0,69.0,1.0,28.0,"New York City, New York",277.31,79.0,3.0,500.0,Rain,light rain,Bronx,Soundview/Castle Hill,10462.0,Bronx,East Concourse/Concourse Village,10451.0


In [6]:
Combined_Data_Fixed = Combined_Data.rename(columns={'Borough_x': 'Pickup_Location', 'Zone_x': 'Pickup_Neighborhood', 'Borough_y': 'Dropoff_Location', 'Zone_y': 'Dropoff_Neighborhood', 'zipcode_x': 'Pickup_ZipCode', 'zipcode_y': 'Dropoff_ZipCode'})

Combined_Data_Fixed = Combined_Data_Fixed.reset_index()

Combined_Data_Fixed = Combined_Data_Fixed.dropna(subset= ['temp'])

Combined_Data_Fixed['temp'] = pd.to_numeric(Combined_Data_Fixed['temp'])

Combined_Data_Fixed['Temp'] = kelvin_to_fahrenheit(Combined_Data_Fixed['temp']).astype(int)

Combined_Data_Fixed['Pickup_City'] = Combined_Data_Fixed["Pickup_Neighborhood"]+', '+Combined_Data_Fixed["Pickup_Location"]

Combined_Data_Fixed['Dropoff_City'] = Combined_Data_Fixed["Dropoff_Neighborhood"]+', '+Combined_Data_Fixed["Dropoff_Location"]

Combined_Data_Fixed = Combined_Data_Fixed[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'SR_Flag', 'Pickup_City', 'Dropoff_City', 'Pickup_ZipCode', 'Dropoff_ZipCode', 'Trip_Duration', 'Temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description']]

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/Cleaned_RandomSamples/')

Combined_Data_Fixed = Combined_Data_Fixed.sort_values(by=['Pickup_DateTime'])

Combined_Data_Fixed.index.name = '#'

Combined_Data_Fixed.to_csv("RideShare_Data_201801.csv", index=True)

print('Clean the combined data and export final file to csv')

Combined_Data_Fixed.head()


Clean the combined data and export final file to csv


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Pickup_City,Dropoff_City,Pickup_ZipCode,Dropoff_ZipCode,Trip_Duration,Temp,humidity,wind_speed,weather_id,weather_main,weather_description
#,,,,,,,,,,,,,,,,
0,2018-12-01 00:00:00,2018-12-01 00:41:27,4.0,80.0,1.0,"Alphabet City, Manhattan","East Williamsburg, Brooklyn",10009.0,11216.0,41.0,39,79.0,3.0,500.0,Rain,light rain
1,2018-12-01 00:00:24,2018-12-01 00:48:55,142.0,17.0,0.0,"Lincoln Square East, Manhattan","Bedford, Brooklyn",10024.0,11216.0,49.0,39,79.0,3.0,500.0,Rain,light rain
2,2018-12-01 00:00:28,2018-12-01 00:09:10,232.0,79.0,0.0,"Two Bridges/Seward Park, Manhattan","East Village, Manhattan",10024.0,10024.0,9.0,39,79.0,3.0,500.0,Rain,light rain
3,2018-12-01 00:00:53,2018-12-01 00:16:45,162.0,107.0,0.0,"Midtown East, Manhattan","Gramercy, Manhattan",10024.0,10024.0,16.0,39,79.0,3.0,500.0,Rain,light rain
4,2018-12-01 00:00:56,2018-12-01 00:29:09,213.0,69.0,1.0,"Soundview/Castle Hill, Bronx","East Concourse/Concourse Village, Bronx",10462.0,10451.0,28.0,39,79.0,3.0,500.0,Rain,light rain
